### This script allows to create a map of the spatial distribution of precipitation averaged over 1983 to 2019 overlaid by cloud band days per year, over the South Pacific.

Before running this script, ensure that you have extracted the cloud bands and have the cloud band files ready. Edit config/config_analysis.yml as needed (eg: data location)

Acknowledgement: the work performed was done by using data from the Global Precipitation Climatology Project and from ERA5.

In [ ]:
# Imports

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.cm import get_cmap
from netCDF4 import Dataset
import numpy as np
import os

import cartopy.crs as ccrs
import cartopy.util as cutil
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

try:
    import colorcet
except ImportError:
    colorcet = None

from cloudbandpy.figure_tools import set_fontsize
from cloudbandpy.io_utilities import load_ymlfile, load_data_from_saved_var_files, subset_latitudes, subset_longitudes
from cloudbandpy.misc import parse_arguments
from cloudbandpy.time_utilities import create_list_of_dates, add_startend_datetime2config
from cloudbandpy.tracking import compute_density

Load GPCP data 

In [ ]:
config_file = os.path.join("../config", "config_analysis.yml")

config = load_ymlfile(config_file, isconfigfile=True)

data_localpath = config["data_localpath"]
filename = "GPCPMON_L3_1983_2019_timemean.nc4"
file_path = os.path.join(data_localpath, filename)
ncfile = Dataset(file_path, "r")
lons = ncfile.variables["lon"][:]
lats = ncfile.variables["lat"][:]
precip = ncfile.variables["sat_gauge_precip"][0]  # mm/day

# Add cyclic point
cdata, clons = cutil.add_cyclic_point(precip, lons)


In [ ]:
def plot_gpcg_precip_cloudbandday(lons, lats, lonscb, latscb, config):
    # Global map of OLR year mean
    set_fontsize(17)
    LON_FORMAT = LongitudeFormatter(zero_direction_label=True, degree_symbol="")
    LAT_FORMAT = LatitudeFormatter(degree_symbol="")
    try:
        # better looking colorblind-proof colormap from colorcet
        cmap = get_cmap("cet_CET_CBL2_r")
    except:
        cmap = get_cmap("viridis")
    #
    fig = plt.figure(figsize=(10, 8))
    ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
    fill = ax.contourf(
        lons,
        lats,
        np.ma.masked_where(cdata == 0, cdata),
        transform=ccrs.PlateCarree(),
        levels=np.arange(0.0, 12.2, 0.2),
        cmap=cmap,
        extend="max",
    )
    cs = ax.contour(
        lonscb,
        latscb,
        np.ma.masked_where(density == 0, density),
        transform=ccrs.PlateCarree(),
        levels=range(0, 56, 7),
        linewidths=1.5,
        colors="#497C3F",
    )
    ax.clabel(cs, cs.levels, inline=True)
    ax.coastlines("50m", color="#404040", linewidth=0.8)
    ax.set_extent(
        (config["lon_west"], config["lon_east"], config["lat_south"], config["lat_north"]), crs=ccrs.PlateCarree()
    )
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=0.6, color="gray", alpha=0.4, linestyle="-")
    gl.top_labels = False
    gl.ylocator = mticker.MultipleLocator(20)
    gl.xlocator = mticker.MultipleLocator(20)
    gl.xformatter = LON_FORMAT
    gl.yformatter = LAT_FORMAT
    #
    im_ratio = cdata.shape[0] / cdata.shape[1]
    cbar = fig.colorbar(fill, orientation="vertical", ticks=range(0, 13, 1), fraction=0.04 * im_ratio)
    cbar.ax.set_yticklabels([el if el % 2 == 0 else "" for el in range(13)])
    cbar.set_label(r"Precipitation rate (mm.day$^{-1}$)")
    return fig


if __name__ == "__main__":
    
    # Make sure that the period for the cloud bands cover the same period as GPCP
    config_copy = config.copy()
    config_copy["startdate"] = "19830101.00"
    config_copy["enddate"] = "20191231.00"
    add_startend_datetime2config(config_copy)

    # Get latitude and longitude of 0.5 degree ERA5 data for the cloud bands
    lons_globe = np.arange(0,360,.5)
    lats_globe = np.arange(90,-90.5,-.5)
    # Get longitudes and latitudes of South Pacific domain
    _, lonsSP = subset_longitudes(lons_globe, config_copy["lon_west"], config_copy["lon_east"])
    _, latsSP = subset_latitudes(lats_globe, config_copy["lat_north"], config_copy["lat_south"])
    
    # Load cloud bands from the South Pacific
    listofdates = create_list_of_dates(config_copy)
    print(listofdates)
    list_of_cloud_bands = load_data_from_saved_var_files(config_copy, varname="list_of_cloud_bands")
    # Compute density for the southern hemisphere
    _, density = compute_density(lats=latsSP, lons=lonsSP, dates=listofdates, list_of_cloud_bands=list_of_cloud_bands)
    # Create figure and save it
    fig = plot_gpcg_precip_cloudbandday(clons, lats, lonsSP, latsSP, config_copy)
    fig.show()
    fig.savefig(
        f"{config_copy['dir_figures']}/map_precip_GPCP_avg_{config_copy['datetime_startdate'].year}-{config_copy['datetime_enddate'].year}_cloudbands7days.png",
        dpi=200,
        bbox_inches="tight",
    )
